## Spark

In [1]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession

def init_spark(app_name: str):
    spark = SparkSession.builder.appName(app_name).getOrCreate()
    sc = spark.sparkContext
    return spark, sc

spark, sc = init_spark('lab3')
sc

<SparkContext master=local[*] appName=lab3>

In [2]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

data_csv = 'Lab3_view_data.csv'
data_rdd = sc.textFile(data_csv)
data_rdd = data_rdd.map(lambda row: row.split(',')) #split by comma
header = data_rdd.first()
data_rdd2 = data_rdd.filter(lambda row: row != header) #remove the header

#Finds the number of viewing events for each device in each day during Prime-Time (20:00-23:00):
prime_time_data = data_rdd2.filter(lambda row: int(row[3])>=200000 and int(row[3])<230000)

#Sorts the data according to views per device:
sorted_data = prime_time_data.map(lambda x:(x[1],1)).reduceByKey( lambda x,y:x+y).sortBy( lambda x: -x[1])
sorted_data.collect()
top_5_devices = (sorted_data.take(5))

#Calculates the number of different dates in the data:
all_dates = data_rdd2.map(lambda x:(x[2], 1)).reduceByKey(lambda x,y: x+y)
all_dates.collect()
all_dates_num = all_dates.count()

#Return the devices and the average amount of viewing events during PrimeTime across all the dates in the data,
#of the 5 devices with the highest averages
for i in top_5_devices:
    print(f"{i[0]}" + f" {i[1]/all_dates_num}")

7.5E+14 97.64285714285714
7.46E+14 11.714285714285714
7.503E+14 9.357142857142858
8.00001E+11 7.5
8.4843E+14 5.5
